In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:53<00:00, 22.9MB/s]
100% 1.06G/1.06G [00:53<00:00, 21.5MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout
from keras.optimizers import Adam,SGD

In [2]:
train_data = keras.utils.image_dataset_from_directory(
    directory='train',
    labels='inferred',
    label_mode='int',
    batch_size = 10,
    image_size=(256,256)
)
test_data = keras.utils.image_dataset_from_directory(
    directory='test',
    labels='inferred',
    label_mode='int',
    batch_size = 10,
    image_size=(256,256)
)

NotFoundError: Could not find directory train

In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_data.map(process)
test_ds = test_data.map(process)

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))
model.add(Conv2D(filters=128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_ds,validation_data=test_ds,epochs=10)

In [53]:
from keras.applications.vgg16 import VGG16

In [54]:
conv_base = VGG16(
    weights='imagenet',
    include_top = False,
    input_shape=(150,150,3)
)

In [55]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [56]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [57]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_5 (Dense)             (None, 256)               2097408   
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [58]:
conv_base.trainable = False

In [59]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [60]:
train_data = keras.utils.image_dataset_from_directory(
    directory='train',
    labels='inferred',
    label_mode='int',
    batch_size = 10,
    image_size=(150,150)
)
test_data = keras.utils.image_dataset_from_directory(
    directory='test',
    labels='inferred',
    label_mode='int',
    batch_size = 10,
    image_size=(150,150)
)
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_data.map(process)
test_ds = test_data.map(process)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [61]:
model.fit(train_ds,validation_data=test_ds,epochs=10)

Epoch 1/10
2000/2000 [==============================] - 89s 44ms/step - loss: 0.2697 - accuracy: 0.8852 - val_loss: 0.2094 - val_accuracy: 0.9114
Epoch 2/10
2000/2000 [==============================] - 82s 41ms/step - loss: 0.2025 - accuracy: 0.9166 - val_loss: 0.2059 - val_accuracy: 0.9104
Epoch 3/10
2000/2000 [==============================] - 88s 44ms/step - loss: 0.1654 - accuracy: 0.9317 - val_loss: 0.1990 - val_accuracy: 0.9146
Epoch 4/10
2000/2000 [==============================] - 83s 41ms/step - loss: 0.1346 - accuracy: 0.9452 - val_loss: 0.2225 - val_accuracy: 0.9090
Epoch 5/10
2000/2000 [==============================] - 88s 44ms/step - loss: 0.1091 - accuracy: 0.9557 - val_loss: 0.2437 - val_accuracy: 0.9118
Epoch 6/10
2000/2000 [==============================] - 88s 44ms/step - loss: 0.0804 - accuracy: 0.9688 - val_loss: 0.2959 - val_accuracy: 0.8964
Epoch 7/10
2000/2000 [==============================] - 88s 44ms/step - loss: 0.0589 - accuracy: 0.9779 - val_loss: 0.3362 -